In [7]:
import pickle
import numpy as np
import re
import keras
from keras.models import Model
from keras.layers import Input, LSTM, Dense,GRU,Dropout

Using from 2 sites:
*Main: https://machinelearningmastery.com/develop-encoder-decoder-model-sequence-sequence-prediction-keras/
*Second https://stackoverflow.com/questions/49477097/keras-seq2seq-word-embedding

In [37]:
def minimal_clean(corpus):
    #Normalize tabs and remove newlines
    no_tabs = str(corpus).replace('\t', ' ').replace('\n', '')

    #Remove all characters except A-Z and a dot.
    alphas_only = re.sub("[^a-zA-Z\.\,\']", " ", no_tabs)

    #Normalize spaces to 1
    multi_spaces = re.sub(" +", " ", alphas_only)
    
    # reduce space between end of sentence
    periods = re.sub(" \.", ".",multi_spaces)
    
    return periods.lower()

In [38]:
query_response = pickle.load(open("../data/query_response_direct.p","rb"))
print(len(query_response))
query_response = [(minimal_clean(q),minimal_clean(r)) for q,r in query_response]

96112


In [39]:
query_response[0]

("pertinent facts age sex male height ' cm weight lbs, has fluctuated in the last six months from to. race caucasian duration of complaint months details below location geographic dallas, tx on body virtually everywhere details below current medications elmiron mg day hydroxyzine mg day trazodone mg day duloxetine mg day mirtazapine mg day gabapentin mg, day tramadol as needed levetiracetam keppra mg day diazepam mg as needed specialists seen urologist, neurologist, psychiatrist, psychologist, gastroenterologist, internist, er diagnostics run two abdominal ct scans one with contrast, one without , chest x ray, lumbar puncture with emergency blood patch following , endoscopy, colonscopy, cystoscopy with hydrodistention, full spinal mri without contrast , at least six full scale blood draws, urinalysis, other 'minor' testsi will try to keep this as brief as possible.last march, my husband z had what he thought initially was a bladder infection following a kidney stone feeling of urgency,

In [41]:
batch_size = 8  # Batch size for training.
epochs = 100  # Number of epochs to train for.
latent_dim = 256  # Latent dimensionality of the encoding space.
num_samples = 2000 # Number of samples to train on.


#Vectorize the data for words
input_texts = []
target_texts = []
input_words = set()
target_words = set()

lines = query_response
# Option 2
for seq1, seq2 in lines[: min(num_samples, len(lines) - 1)]:
    input_text, target_text = seq1, seq2
    # We use "tab" as the "start sequence" character
    # for the targets, and "\n" as "end sequence" character.
    target_text = '<BOS> ' + target_text + ' <EOS>' #'<BOS>' + target_text + '<EOS>'
    input_texts.append(input_text)
    target_texts.append(target_text)
    
    # Only one input word
    input_text_list = input_text.split()
    if len(input_text_list) == 1:
        input_words.add(input_text_list[0])
    # Multiple input words
    else:
        for word in input_text_list:
            input_words.add(word)
                
    # For right now we are only looking at 1 target...the medication
    target_text_list = target_text.split()
    for word in target_text_list:
        target_words.add(word)

# Option 2
input_words = sorted(list(input_words))
target_words = sorted(list(target_words))
num_encoder_tokens = len(input_words)
num_decoder_tokens = len(target_words)
# set max input sequence (may need to pad...)
max_encoder_seq_length = max([len(txt) for txt in input_texts])
max_decoder_seq_length = max([len(txt) for txt in target_texts])

print('Number of samples:', len(input_texts))
print('Number of unique input tokens:', num_encoder_tokens)
print('Number of unique output tokens:', num_decoder_tokens)
print('Max sequence length for inputs:', max_encoder_seq_length)
print('Max sequence length for outputs:', max_decoder_seq_length)

Number of samples: 2000
Number of unique input tokens: 15875
Number of unique output tokens: 12976
Max sequence length for inputs: 11922
Max sequence length for outputs: 9575


In [18]:
batch_size = 8  # Batch size for training.
epochs = 100  # Number of epochs to train for.
latent_dim = 256  # Latent dimensionality of the encoding space.

# Define an input sequence and process it.
encoder_inputs = Input(shape=(None, num_encoder_tokens))
encoder = LSTM(latent_dim, return_state=True,dropout=.1,recurrent_dropout=0.1)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)
# We discard `encoder_outputs` and only keep the states.
encoder_states = [state_h, state_c]

NameError: name 'num_encoder_tokens' is not defined

In [ ]:
n_input =  # The cardinality of the input sequence, e.g. number of features, words, or characters for each time step.
n_output = # The cardinality of the output sequence, e.g. number of features, words, or characters for each time step.
n_units = 256 # The number of cells to create in the encoder and decoder models, e.g. 128 or 256.


encoder_inputs = Input(shape=(sentenceLength,), name="Encoder_input")
encoder = LSTM(n_units, return_state=True, name='Encoder_lstm') 
Shared_Embedding = Embedding(output_dim=embedding, input_dim=vocab_size, name="Embedding") 
word_embedding_context = Shared_Embedding(encoder_inputs) 
encoder_outputs, state_h, state_c = encoder(word_embedding_context) 
encoder_states = [state_h, state_c] decoder_lstm = LSTM(n_units, return_sequences=True, return_state=True, name="Decoder_lstm") 
word_embedding_answer = Shared_Embedding(decoder_inputs) decoder_outputs, _, _ = decoder_lstm(word_embedding_answer, initial_state=encoder_states) 
decoder_dense = Dense(vocab_size, activation='softmax', name="Dense_layer") 
decoder_outputs = decoder_dense(decoder_outputs) 
model = Model([encoder_inputs, decoder_inputs], decoder_outputs) 
encoder_model = Model(encoder_inputs, encoder_states) 
decoder_state_input_h = Input(shape=(n_units,), name="H_state_input") 
decoder_state_input_c = Input(shape=(n_units,), name="C_state_input") 
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c] 
decoder_outputs, state_h, state_c = decoder_lstm(word_embedding_answer, initial_state=decoder_states_inputs) 
decoder_states = [state_h, state_c] 
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = Model([decoder_inputs] + decoder_states_inputs, [decoder_outputs] + decoder_states)

In [42]:
from random import randint
from numpy import array
from numpy import argmax
from numpy import array_equal
from keras.utils import to_categorical
from keras.models import Model
from keras.layers import Input
from keras.layers import LSTM
from keras.layers import Dense

# generate a sequence of random integers
def generate_sequence(length, n_unique):
	return [randint(1, n_unique-1) for _ in range(length)]

# prepare data for the LSTM
def get_dataset(n_in, n_out, cardinality, n_samples):
	X1, X2, y = list(), list(), list()
	for _ in range(n_samples):
		# generate source sequence
		source = generate_sequence(n_in, cardinality)
		# define padded target sequence
		target = source[:n_out]
		target.reverse()
		# create padded input target sequence
		target_in = [0] + target[:-1]
		# encode
		src_encoded = to_categorical([source], num_classes=cardinality)
		tar_encoded = to_categorical([target], num_classes=cardinality)
		tar2_encoded = to_categorical([target_in], num_classes=cardinality)
		# store
		X1.append(src_encoded)
		X2.append(tar2_encoded)
		y.append(tar_encoded)
	return array(X1), array(X2), array(y)

# returns train, inference_encoder and inference_decoder models
def define_models(n_input, n_output, n_units):
	# define training encoder
	encoder_inputs = Input(shape=(None, n_input))
	encoder = LSTM(n_units, return_state=True)
	encoder_outputs, state_h, state_c = encoder(encoder_inputs)
	encoder_states = [state_h, state_c]
	# define training decoder
	decoder_inputs = Input(shape=(None, n_output))
	decoder_lstm = LSTM(n_units, return_sequences=True, return_state=True)
	decoder_outputs, _, _ = decoder_lstm(decoder_inputs, initial_state=encoder_states)
	decoder_dense = Dense(n_output, activation='softmax')
	decoder_outputs = decoder_dense(decoder_outputs)
	model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
	# define inference encoder
	encoder_model = Model(encoder_inputs, encoder_states)
	# define inference decoder
	decoder_state_input_h = Input(shape=(n_units,))
	decoder_state_input_c = Input(shape=(n_units,))
	decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
	decoder_outputs, state_h, state_c = decoder_lstm(decoder_inputs, initial_state=decoder_states_inputs)
	decoder_states = [state_h, state_c]
	decoder_outputs = decoder_dense(decoder_outputs)
	decoder_model = Model([decoder_inputs] + decoder_states_inputs, [decoder_outputs] + decoder_states)
	# return all models
	return model, encoder_model, decoder_model

# generate target given source sequence
def predict_sequence(infenc, infdec, source, n_steps, cardinality):
	# encode
	state = infenc.predict(source)
	# start of sequence input
	target_seq = array([0.0 for _ in range(cardinality)]).reshape(1, 1, cardinality)
	# collect predictions
	output = list()
	for t in range(n_steps):
		# predict next char
		yhat, h, c = infdec.predict([target_seq] + state)
		# store prediction
		output.append(yhat[0,0,:])
		# update state
		state = [h, c]
		# update target sequence
		target_seq = yhat
	return array(output)

# decode a one hot encoded string
def one_hot_decode(encoded_seq):
	return [argmax(vector) for vector in encoded_seq]

# configure problem
n_features = 50 + 1
n_steps_in = 6
n_steps_out = 3
# define model
train, infenc, infdec = define_models(n_features, n_features, 128)
train.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['acc'])
# generate training dataset
X1, X2, y = get_dataset(n_steps_in, n_steps_out, n_features, 100000)
print(X1.shape,X2.shape,y.shape)
# train model
train.fit([X1, X2], y, epochs=1)
# evaluate LSTM
total, correct = 100, 0
for _ in range(total):
	X1, X2, y = get_dataset(n_steps_in, n_steps_out, n_features, 1)
	target = predict_sequence(infenc, infdec, X1, n_steps_out, n_features)
	if array_equal(one_hot_decode(y[0]), one_hot_decode(target)):
		correct += 1
print('Accuracy: %.2f%%' % (float(correct)/float(total)*100.0))
# spot check some examples
for _ in range(10):
	X1, X2, y = get_dataset(n_steps_in, n_steps_out, n_features, 1)
	target = predict_sequence(infenc, infdec, X1, n_steps_out, n_features)
	print('X=%s y=%s, yhat=%s' % (one_hot_decode(X1[0]), one_hot_decode(y[0]), one_hot_decode(target)))

(100000, 1, 6, 51) (100000, 1, 3, 51) (100000, 1, 3, 51)


ValueError: Error when checking input: expected input_1 to have 3 dimensions, but got array with shape (100000, 1, 6, 51)